In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from API_keys import g_key

In [2]:
weather_data = pd.read_csv("Resources/Homework-06-Python-APIs_output_data_cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hithadhoo,-0.60,73.08,82.35,77,88,12.95,MV,1599524962
1,Constitución,-35.33,-72.42,50.09,83,87,5.91,CL,1599524962
2,Martaban,16.53,97.61,78.13,93,100,4.85,MM,1599524962
3,Yellowknife,62.46,-114.35,50.00,81,75,16.11,CA,1599524775
4,Baruun-Urt,46.68,113.28,57.07,70,1,9.48,MN,1599524963
...,...,...,...,...,...,...,...,...,...
562,Bima,-8.47,118.72,86.70,54,10,4.21,ID,1599525245
563,Santiago del Estero,-27.80,-64.26,64.22,53,80,6.96,AR,1599525074
564,Yabēlo,4.88,38.08,58.15,86,66,6.80,ET,1599525246
565,Sampang,-7.19,113.24,82.09,72,34,9.10,ID,1599525246


In [3]:
#Configure gmaps.
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.
gmaps.configure(api_key=g_key)
locations = weather_data[['Lat', 'Lng']]
humidity = weather_data['Humidity']

In [4]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.
conditions_df = weather_data.loc[(weather_data['Wind Speed'] < 10) & (weather_data['Cloudiness'] == 0) & (weather_data['Max Temp'] >= 70) & (weather_data['Max Temp'] <= 80)].dropna()

conditions_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,Al Bardīyah,31.76,25.09,73.45,71,0,5.64,LY,1599524966
29,Pitimbu,-7.47,-34.81,71.60,88,0,4.70,BR,1599524972
60,Lata,40.16,-8.33,75.99,40,0,1.01,PT,1599524985
116,Ajdabiya,30.76,20.23,72.95,49,0,9.60,LY,1599525010
124,Santa Maria da Vitória,-13.39,-44.19,76.08,39,0,7.11,BR,1599525015


In [6]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.


hotel_df = conditions_df.loc[:,['City','Country', 'Lat', 'Lng']]
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
13,Al Bardīyah,LY,31.76,25.09,
29,Pitimbu,BR,-7.47,-34.81,
60,Lata,PT,40.16,-8.33,
116,Ajdabiya,LY,30.76,20.23,
124,Santa Maria da Vitória,BR,-13.39,-44.19,


In [7]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

parameters = {'type': 'hotel','keyword': 'hotel', 'radius': 5000,'key': g_key}

In [8]:
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city_name = row['City']
    
    parameters['location'] = f'{lat},{lng}'

    print(f'Retrieving Results for Index {index}: {city_name}.')
    response = requests.get(base_url, params=parameters).json()
    
    results = response['results']

    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except (KeyError, IndexError):
        print('Missing field/result... skipping.')
        
    print('--------------------------')
    time.sleep(1)
    
print('-------End of Search-------')

Retrieving Results for Index 13: Al Bardīyah.
Closest hotel in Al Bardīyah is Hotel Burdy.
--------------------------
Retrieving Results for Index 29: Pitimbu.
Closest hotel in Pitimbu is Pousada O Nativo.
--------------------------
Retrieving Results for Index 60: Lata.
Missing field/result... skipping.
--------------------------
Retrieving Results for Index 116: Ajdabiya.
Closest hotel in Ajdabiya is Amal Libya Hotel.
--------------------------
Retrieving Results for Index 124: Santa Maria da Vitória.
Closest hotel in Santa Maria da Vitória is Hotel Portal do Corrente.
--------------------------
Retrieving Results for Index 187: Porangatu.
Closest hotel in Porangatu is Lagoa Hotel Porangatu.
--------------------------
Retrieving Results for Index 196: Tirat Carmel.
Closest hotel in Tirat Carmel is מלון קדם Kedem Hotel.
--------------------------
Retrieving Results for Index 207: Pimenteiras.
Closest hotel in Pimenteiras is Posto e Pousada JJ.
--------------------------
Retrieving Res

In [9]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
13,Al Bardīyah,LY,31.76,25.09,Hotel Burdy
29,Pitimbu,BR,-7.47,-34.81,Pousada O Nativo
60,Lata,PT,40.16,-8.33,
116,Ajdabiya,LY,30.76,20.23,Amal Libya Hotel
124,Santa Maria da Vitória,BR,-13.39,-44.19,Hotel Portal do Corrente


In [10]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))